In [3]:
import csv
import json
import os
import re
import string

import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI

#### Function to generate batch data

In [4]:
import os

import openai
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get the API key from the environment variable
api_key = os.getenv("OPENAI_API_KEY")
print(f"Loaded API Key: {api_key}")

# Check if the API key was loaded correctly
if not api_key:
    raise ValueError("API Key not found in environment variables!")


# Set the OpenAI API key
openai.api_key = api_key

Loaded API Key: sk-afklp7QTf3QSs-7f7ufW8E-9c-YjY64thYzjuHk5ThT3BlbkFJCbUZuS8Dq8QlRMKRBO_uHsho_ujq9OEZ6OAnzBIb8A


In [47]:
def generate_openai_requests_json(input_file, chunk_size):
    system_prompt = """
    You are a multiclass text classifier. Comments should be taken as they are, without inferring tone or context. 
    Step 1: Group Sensitivity Determine if the text refers to a group defined by any of the following characteristics: 
    Nationality, Race, Ethnicity, Religion, Gender, Sexual orientation, Disability, Skin color, Age, Generational group, 
    Socio-economic status, Immigration status. Sensitive Group: If the text explicitly mentions a group belonging to any of these characteristics, 
    classify it as Sensitive Group: Yes. Non-Sensitive Group: If the text does not mention any of these characteristics, classify it as Sensitive Group: 
    No. Step 2: Language Intensity Classification For Sensitive Groups (Hate) Hate 1: Statements containing bias or generalizations without exclusion or incitement. 
    Hate 2: Statements that add exclusion or denial of rights to Hate 1. This includes explicit or implicit denial of access, rights, opportunities, 
    or services based on the group's characteristics. Hate 3: Statements that incite harm or make threats. For Non-Sensitive Groups (Toxic) 
    Toxic 1: Mild complaints or disrespect without direct insults or threats. Toxic 2: Clear insults, sarcasm, or mocking. Toxic 3: Harassment or encouragement of harm. 
    For Any Groups No Hate/Toxic: Neutral or constructive comments, including mild criticism or playful sarcasm. Output Format Sensitive Group: Yes/No Classification: (e.g., "No Hate/Toxic", "Hate 1", etc.) Examples: 
    Example 1: "Expats in Singapore always expect preferential treatment." Sensitive Group: Yes Classification: Hate 1 Example 2: "I don’t let Indians in 
    because they are smelly." Sensitive Group: Yes Classification: Hate 2 Example 3: "Gen Zs don’t belong here. I’ll kill them if they stay." Sensitive Group: Yes Classification: 
    Hate 3 Classify this comment: 
    """

    # Read input CSV file
    df = pd.read_csv(input_file)
    
    # Initialize lists to store results
    sensitive_groups = []
    classifications = []

    # Process each comment individually
    for index, row in df.iterrows():
        comment = row['text']
        
        requests = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": comment}
        ]

        try:
            # Sending request for the current comment
            response = openai.chat.completions.create(
                model="gpt-4o-mini",  # Specify your model here
                messages=requests,
                max_tokens=100,
                n=1,  # Generate one response for the current comment
            )

            # Process response
            choice = response.choices[0]
            result = choice.message.content.strip()
            # Assuming the output is in the format: "Sensitive Group: Yes Classification: Hate 1"
            sensitive_group, classification = result.split("Classification:")
            sensitive_groups.append(sensitive_group.replace("Sensitive Group:", "").strip())
            classifications.append(classification.strip())

        except Exception as e:
            # Append None if there's an error for the current comment
            sensitive_groups.append(None)
            classifications.append(None)

        # Save results after every `chunk_size` comments
        if (index + 1) % chunk_size == 0:
            # Append results to the original DataFrame
            start_index = index - chunk_size + 1
            df.loc[start_index:index, 'Sensitive Group'] = sensitive_groups[start_index:index + 1]
            df.loc[start_index:index, 'Classification'] = classifications[start_index:index + 1]

            # Save the updated DataFrame back to the same CSV file
            df.to_csv(input_file, index=False)

    # Save any remaining results if they exist after the loop
    if len(sensitive_groups) > len(df):
        # Append any remaining results
        df['Sensitive Group'] = sensitive_groups
        df['Classification'] = classifications
        df.to_csv(input_file, index=False)




#### Creating the batch data

In [49]:
## run this
input_file= "C:\\Users\\Yi Jing\\Documents\\Y4S1\\dsa4264\\DSA4264-Detoxify\\predicted_labels_output_2023 - Copy.csv"
generate_openai_requests_json(input_file,chunk_size=100) # change chunk size if needed, happy labelling! 